In [52]:
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# CSV 파일에서 데이터 불러오기
data = pd.read_csv('doremi_fre.csv')

# 피쳐와 레이블 분리
X = data.iloc[:, 0].values.reshape(47, 1)
y = data.iloc[:, 1]

# 학습 데이터와 검증 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001)

# 의사결정나무 모델 생성 및 학습
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)


ys, sr = librosa.load('user_voice3high.wav')
start = int(1 * sr)
end = int(2 * sr)
f0, voiced_flag, voiced_probs = librosa.pyin(ys[start:end], fmin=librosa.note_to_hz('C1'), fmax=librosa.note_to_hz('B4'))
valid_pitch = f0[~np.isnan(f0)]

pitch_mean = np.nanmean(valid_pitch)
print(pitch_mean)

pitch_mean_reshaped = pitch_mean.reshape(-1, 1)  # reshape
print(pitch_mean_reshaped)


# 분류 결과 출력
predicted = dt.predict(pitch_mean_reshaped)  # 사용할 변수 재할당
print('Predicted Label:', predicted) 

206.8823341587939
[[206.88233416]]
Predicted Label: ['G#3']
